In [32]:
import pandas as pd
import sqlite3

- create a connection to the database using the library sqlite3

In [33]:
conn = sqlite3.connect('../data/checking-logs.sqlite.sqlite')

- get the schema of the table test

In [34]:
pd.io.sql.read_sql('PRAGMA table_info(checker)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


- get only the first 10 rows of the table test to check what the table looks like

In [35]:
pd.io.sql.read_sql('SELECT * FROM checker LIMIT 10', conn)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


- find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query
    - do this by joining the table with the table deadlines
    - the difference should be displayed in hours
    - do not take the lab ’project1’ into account, it has longer deadlines and will be an outlier
    - the value should be stored in the dataframe df_min with the corresponding uid

In [36]:
df_min = pd.io.sql.read_sql(
"""
SELECT uid, cast((julianday(test.checker_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS diff
FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs
WHERE NOT test.labname = 'project1'
ORDER BY diff LIMIT 1
""",
conn).rename({'diff': 'MIN(diff)'}, axis=1)
df_min

,uid,MIN(diff)
0,user_30,-202


- do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [37]:
df_max = pd.io.sql.read_sql(
"""
SELECT uid, cast((julianday(test.checker_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS diff
FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs
WHERE NOT test.labname = 'project1'
ORDER BY diff DESC LIMIT 1
""",
conn).rename({'diff': 'MAX(diff)'}, axis=1)
df_max

,uid,MAX(diff)
0,user_25,-2


- do the same thing but for the average, using on lyonequery, this time your data frame should not include the uid column, and the dataframe name is df_avg

In [38]:
df_avg = pd.io.sql.read_sql(
"""
SELECT avg(diff)
FROM (
SELECT cast((julianday(test.checker_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS diff
FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs
WHERE NOT test.labname = 'project1')
""",
conn).rename({'avg(diff)': 'AVG(diff)'}, axis=1)
df_avg

,AVG(diff)
0,-89.125


- we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference
    - using only one query, create a table with the columns: uid, avg_diff, pageviews
    - uid is the uids that exist in the test
    - avg_diff is the average delta between the first commit and the lab deadline per user
    - pageviews is the number of Newsfeed visits per user
    - do not take the lab ’project1’ into account
    - store it to the dataframe views_diff

In [39]:
views_diff = pd.io.sql.read_sql(
"""
SELECT diff.uid, avg_diff, pageviews
FROM (
SELECT uid, avg(cast((julianday(test.checker_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 as integer)) as avg_diff
FROM test left join deadlines on test.labname = deadlines.labs
WHERE NOT test.labname = 'project1'GROUP BY uid) AS diff LEFT JOIN (SELECT uid, count(*) AS pageviews
FROM  pageviews GROUP BY uid) AS views ON diff.uid = views.uid
""",
conn)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


- use the Pandas method corr() to calculate the correlation coefficient between the number of pageviews and the difference  

In [40]:
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


- close the connection

In [41]:
conn.close()